In [ ]:
Enviroment Setup 

In [1]:
import os
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.models import resnet50
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import csv

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt

In [ ]:
Data pre_processing

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

target_size = (64, 64)  
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    '/kaggle/input/arabic-letters-classification/Final_Arabic_Alpha_dataset/Final_Arabic_Alpha_dataset/train/',
    target_size=target_size,
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Load the validation data
validation_generator = datagen.flow_from_directory(
    '/kaggle/input/arabic-letters-classification/Final_Arabic_Alpha_dataset/Final_Arabic_Alpha_dataset/train/',
    target_size=target_size,
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 34064 images belonging to 65 classes.
Found 8495 images belonging to 65 classes.


Model Defenition and Summary 

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the model architecture
model = models.Sequential()

# Add Convolutional layers
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Flatten the output for the fully connected layers
model.add(layers.Flatten())

# Add Fully Connected layers
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(65, activation='softmax'))  # Assuming 65 classes

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Display the model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 31, 31, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 14, 14, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 6, 6, 128)         0

Model optmization 

In [4]:
from tensorflow.keras.optimizers import Adam
# Use the Adam optimizer with default parameters
optimizer = Adam()
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

Train the model 

In [5]:
epochs=20
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[early_stop]
)


Epoch 1/20
1064/1064 [==============================] - 267s 249ms/step - loss: 3.9320 - accuracy: 0.0538 - val_loss: 3.3470 - val_accuracy: 0.1498
Epoch 2/20
1064/1064 [==============================] - 192s 181ms/step - loss: 2.8542 - accuracy: 0.2357 - val_loss: 2.4796 - val_accuracy: 0.3233
Epoch 3/20
1064/1064 [==============================] - 178s 168ms/step - loss: 2.2853 - accuracy: 0.3605 - val_loss: 2.1268 - val_accuracy: 0.4057
Epoch 4/20
1064/1064 [==============================] - 191s 180ms/step - loss: 2.0154 - accuracy: 0.4301 - val_loss: 1.9439 - val_accuracy: 0.4412
Epoch 5/20
1064/1064 [==============================] - 194s 182ms/step - loss: 1.8591 - accuracy: 0.4665 - val_loss: 1.8009 - val_accuracy: 0.4886
Epoch 6/20
1064/1064 [==============================] - 181s 170ms/step - loss: 1.7323 - accuracy: 0.4987 - val_loss: 1.7319 - val_accuracy: 0.5025
Epoch 7/20
1064/1064 [==============================] - 194s 182ms/step - loss: 1.6515 - accuracy: 0.5232 - val_

Save Model 

In [6]:
model.save("/kaggle/working/saved_model/")


Load Model 

In [7]:
from tensorflow.keras.models import load_model
# Load the saved model
loaded_model = load_model("/kaggle/working/saved_model/")

Prediction 

In [37]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import time

# Assuming test_images_dir is the directory containing your test images
test_images_dir = "/kaggle/input/arabic-letters-classification/Final_Arabic_Alpha_dataset/Final_Arabic_Alpha_dataset/test/"

# Create an empty list to store predictions
predictions_list = []

# Set the maximum number of images to process
max_images = 10610

# Set the interval for progress updates
progress_interval = 100

# Create a counter to track the number of processed images
image_count = 0

# Start the timer
start_time = time.time()

# Iterate through the images in the test directory
for image_name in os.listdir(test_images_dir):
    if image_count >= max_images:
        break  # Exit the loop when the maximum number of images is reached

    image_path = os.path.join(test_images_dir, image_name)

    # Check if the file is an image (you might need to adjust the list of valid extensions)
    if image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
        try:
            # Load and preprocess the image
            img = load_img(image_path, target_size=(64, 64))
            img_array = img_to_array(img)
            img_array = img_array / 255.0  # Normalize pixel values to be between 0 and 1
            img_array = np.expand_dims(img_array, axis=0)

            # Make predictions using your model
            predictions = loaded_model.predict(img_array)

            # Assuming you have 65 classes, adjust as needed
            predicted_class = np.argmax(predictions)

            # Append the result to the list
            predictions_list.append({"ID": image_name, "Label": predicted_class})

            # Increment the image count
            image_count += 1

            # Print progress every 'progress_interval' images
            if image_count % progress_interval == 0:
                print(f"Processed {image_count} images")

        except Exception as e:
            print(f"Error processing image {image_name}: {str(e)}")

# End the timer
end_time = time.time()

# Calculate the total processing time
total_time = end_time - start_time

# Print the total processing time
print(f"Total processing time for {image_count} images: {total_time:.2f} seconds")

# Create a DataFrame from the list
predictions_df = pd.DataFrame(predictions_list)

# Save the DataFrame to a CSV file
csv_file_path = "/kaggle/working/predictions.csv"
predictions_df.to_csv(csv_file_path, index=False)

# Display the DataFrame
print(predictions_df)


1/1 [==============================] - 0s 24ms/step
Processed 100 images
1/1 [==============================] - 0s 24ms/step
Processed 200 images
1/1 [==============================] - 0s 25ms/step
Processed 300 images
1/1 [==============================] - 0s 25ms/step
Processed 400 images
1/1 [==============================] - 0s 33ms/step
Processed 500 images
1/1 [==============================] - 0s 26ms/step
Processed 600 images
1/1 [==============================] - 0s 24ms/step
Processed 700 images
1/1 [==============================] - 0s 24ms/step
Processed 800 images
1/1 [==============================] - 0s 25ms/step
Processed 900 images
1/1 [==============================] - 0s 26ms/step
Processed 1000 images
1/1 [==============================] - 0s 26ms/step
Processed 1100 images
1/1 [==============================] - 0s 24ms/step
Processed 1200 images
1/1 [==============================] - 0s 24ms/step
Processed 1300 images
1/1 [==============================] - 0s 25ms/